In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
mnist.data.shape

C:\Users\Family PC\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)

In [2]:
mnist.data

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
mnist.target

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [4]:
import tensorflow as tf

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target, random_state = 0)

In [6]:
input_width = 28
input_height = 28
input_channel = 1
input_pixel = 784

n_conv1 = 32
n_conv2 = 64

stride_conv1 = 1
stride_conv2 = 1

conv1_k = 5
conv2_k = 5

max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden_layer = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [7]:
weights = {
    "wc1": tf.Variable(tf.compat.v1.random_normal([conv1_k,conv1_k, input_channel,n_conv1])),
    "wc2": tf.Variable(tf.compat.v1.random_normal([conv2_k,conv2_k, n_conv1,n_conv2])),
    "whl": tf.Variable(tf.compat.v1.random_normal([input_size_to_hidden_layer, n_hidden])),
    "wo" : tf.Variable(tf.compat.v1.random_normal([n_hidden, n_out]))
}

In [8]:
biases = {
    "bc1": tf.Variable(tf.compat.v1.random_normal([n_conv1])),
    "bc2": tf.Variable(tf.compat.v1.random_normal([n_conv2])),
    "bhl": tf.Variable(tf.compat.v1.random_normal([n_hidden])),
    "bol": tf.Variable(tf.compat.v1.random_normal([n_out])),
}

In [9]:
def conv(x, weights, biases, strides):
    out = tf.nn.conv2d(x, weights, padding = "SAME", strides = [1,strides, strides, 1])
    out = tf.nn.bias_add(out, biases)
    out = tf.nn.relu(out)
    return out

In [10]:
def maxpool(x, k =2):
    return tf.compat.v1.nn.max_pool(x, padding = "SAME", strides = [1,k,k,1], ksize = [1,k,k,1])

In [11]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape = [ -1, input_height, input_width, input_channel])
    conv1 = conv(x, weights['wc1'], biases['bc1'],stride_conv2)
    conv1_pool = maxpool(conv1, max_pool1_k)
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'],stride_conv2)
    conv2_pool = maxpool(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden_layer])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['whl']), biases['bhl'])
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bol'])
    return output

In [12]:
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
x = tf.compat.v1.placeholder("float", [None, input_pixel])
y = tf.compat.v1.placeholder(tf.int32, [None, n_out])
pred = cnn(x , weights, biases)

RuntimeError: resource: Attempting to capture an EagerTensor without building a function.

In [ ]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))

In [ ]:
# optimizer

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [ ]:
import tensorflow as tf
with tf.compat.v1.Session() as sess:
    tf.compat.v1.global_variables_initializer()

In [ ]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.next_batch(batch_size)
        c, _ = tf.compat.v1.Session.run([const, optimize], feed_dict = {x: batch_x, y:batch_y})
        total_cost+=c
    print(total_cost)